In [37]:
import csv
import pandas as pd
import glob
import re
import pandas_profiling
from pandas_profiling import ProfileReport

##### Clean up indexAtoZ file from mass.gov 
https://www.mass.gov/lists/district-court-complaint-manual

In [38]:
# script for removing commas and then inserting commas on column splits for this dataset

# with open("../data/categorizeCharges/input.csv", 'r') as input_csv:
#     csv_reader = csv.reader(input_csv)
    
#     with open("../data/categorizeCharges/output.csv", 'w') as output_csv:
#         csv_writer = csv.writer(output_csv)
        
#         for line in csv_reader:
#             lineStr = ""
#             lineStr = lineStr.join(line)
#             lineStr = lineStr.replace(",", "", 5)
#             firstSpace = lineStr.find(" ")
#             lineStr = lineStr[:firstSpace]+","+lineStr[firstSpace:]
#             if(lineStr.find("§") < 0):
#                 continue
#             secondSpace = lineStr.find(" ", lineStr.find("§")+3)
#             lineStr = lineStr[:secondSpace]+","+lineStr[secondSpace:]
#             thirdSpace = lineStr.find(" ", secondSpace+2)
#             lineStr = lineStr[:thirdSpace]+","+lineStr[thirdSpace:]
#             fourthSpace = lineStr.find(" ", thirdSpace+2)
#             lineStr = lineStr[:fourthSpace]+","+lineStr[fourthSpace:]
#             fifthSpace = lineStr.find(" ", fourthSpace+2)
#             lineStr = lineStr[:fifthSpace]+","+lineStr[fifthSpace:]
#             line[0] = lineStr
#             csv_writer.writerow(line)

#             print(line)

In [39]:
# script for stripping quotes and new line from each row after manual processing of dataset

# with open("../data/categorizeCharges/converted_complaint_manual_index_AtoZ.csv", 'r') as input2_csv:
    
#     with open("../data/categorizeCharges/output2.csv", 'w') as output2_csv:
#         for line in input2_csv:
#             print(line)
#             line = line.strip('”"“\n')
#             print(line)
#             output2_csv.write(line+'\n')

In [40]:
#find the columns that appear in both lists

# def colIntersect(col1,col2):
#     col2Set = set()
#     colList = []
    
#     for c2 in col2:
#         col2Set.add(c2.lower())
    
#     for c1 in col1:
#         c1lower = c1.lower()
#         if(c1lower in col2Set):
#             colList.append(c1lower)
#     return colList
    
    

##### Renamed output from above to output2.csv and import ast indexAtoZ_df for further cleaning

In [41]:
indexAtoZ_df = pd.read_csv("../data/categorizeCharges/output2.csv", quotechar="'")
indexAtoZ_df.head()
indexAtoZ_df.shape

(5812, 6)

In [52]:
# expOfCodes_df = pd.read_csv("../data/categorizeCharges/explanation_of_codes.csv")
# expOfCodes_df

In [43]:
#get first row of unique offense code 
#this removes duplicate rows where offense is the same but effective dates are different

df_unique_offense = indexAtoZ_df.drop_duplicates(subset="Offense Code", keep='first', inplace=False)
df_unique_offense.head()


,Offense Code,Offense,Effective Dates(s),F/M,Stats,Fine
0,265/13A/B,A&B c265 §13A,10/1/43-5/21/02,M+,09,A
2,265/13K/F,A&B ON +60/DISABLED c265 §13K(a½),4/11/05,F,09,A
3,265/13K/A,A&B ON +60/DISABLED WITH INJURY c265 §13K(b),3/17/96,F,09,A
4,265/13K/B,A&B ON +60/DISABLED WITH SERIOUS INJURY c265 §...,3/17/96,F,09,A
5,265/13I/B,A&B ON AMBULANCE PERSONNEL c265 §13I,1/31/90-9/29/10,M+,09,A


In [44]:
# cha

df_unique_offense["OffenseMatch"] = df_unique_offense["Offense"].str.upper()
df_unique_offense["OffenseMatch"] = df_unique_offense["OffenseMatch"].str.replace('\s+|Â|[^0-9a-zA-Z]','', regex=True)
df_unique_offense.head()

/home/linna/Apps/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/linna/Apps/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Offense Code,Offense,Effective Dates(s),F/M,Stats,Fine,OffenseMatch
0,265/13A/B,A&B c265 §13A,10/1/43-5/21/02,M+,09,A,ABC26513A
2,265/13K/F,A&B ON +60/DISABLED c265 §13K(a½),4/11/05,F,09,A,ABON60DISABLEDC26513KA
3,265/13K/A,A&B ON +60/DISABLED WITH INJURY c265 §13K(b),3/17/96,F,09,A,ABON60DISABLEDWITHINJURYC26513KB
4,265/13K/B,A&B ON +60/DISABLED WITH SERIOUS INJURY c265 §...,3/17/96,F,09,A,ABON60DISABLEDWITHSERIOUSINJURYC26513KC
5,265/13I/B,A&B ON AMBULANCE PERSONNEL c265 §13I,1/31/90-9/29/10,M+,09,A,ABONAMBULANCEPERSONNELC26513I


In [45]:
clean_middlesex_df = pd.read_csv("../data/cleaned/clean_middlesex.csv", low_memory=False)
clean_middlesex_df.columns
clean_middlesex_df.shape
clean_middlesex_df.head()

,Case Number,Offense Date,Court Location,Charge,Charge/Crime Type,Disposition,Chapter,Section,Paragraph,JuvenileC,years_since_offense,sex,murder,Expungeable,Offenses_per_case,Inc_Expungeable_Attempts_Are,guilty,Incident_Guilty_or_missing
0,14-01-479818,2013-12-30,SOM,"LICENSE SUSPENDED, OP MV WITH c90 Â§23",Drugs/Distribution/Possession with Intent,GUILTY FILED,90,23,D,False,6.7,0,0.0,Yes,2,True,True,True
1,14-01-479826,2013-12-29,LOW,"LICENSE SUSPENDED, OP MV WITH c90 Â§23",Drugs/Possession,NOLLE PROSEQUI,90,23,D,False,6.7,0,0.0,Yes,6,True,False,False
2,14-01-479854,2013-12-31,SOM,"LICENSE SUSPENDED, OP MV WITH c90 Â§23",Motor Vehicle Other,DISMISSED W/O PREJUDICE,90,23,D,False,6.7,0,0.0,Yes,3,True,False,False
3,14-01-479887,2013-12-28,AYE,"LICENSE SUSPENDED, OP MV WITH c90 Â§23",Motor Vehicle Other,DISMISSED W/O PREJUDICE,90,23,D,False,6.7,0,0.0,Yes,2,True,False,True
4,14-01-479888,2013-05-19,CON,"LICENSE SUSPENDED, OP MV WITH c90 Â§23",Motor Vehicle Other,DISMISSED BY FINES,90,23,D,False,7.3,0,0.0,Yes,2,True,False,True


In [46]:
clean_middlesex_df["OffenseMatch"] = clean_middlesex_df["Charge"].str.upper()
clean_middlesex_df["OffenseMatch"] = clean_middlesex_df["OffenseMatch"].str.replace('\s+|Â|[^0-9a-zA-Z]','', regex=True)

clean_middlesex_df[["Case Number","OffenseMatch"]]

,Case Number,OffenseMatch
0,14-01-479818,LICENSESUSPENDEDOPMVWITHC9023
1,14-01-479826,LICENSESUSPENDEDOPMVWITHC9023
2,14-01-479854,LICENSESUSPENDEDOPMVWITHC9023
3,14-01-479887,LICENSESUSPENDEDOPMVWITHC9023
4,14-01-479888,LICENSESUSPENDEDOPMVWITHC9023
...,...,...
387526,19-12-661383,LARCBYCHECK
387527,19-12-661396,LARCBYCHECK
387528,19-12-661398,LARCBYCHECK
387529,19-12-661398,LARCBYCHECK


In [47]:
merged_df = clean_middlesex_df.merge(df_unique_offense[["F/M", "OffenseMatch", "Offense Code"]], how='left', on='OffenseMatch', )
merged_df.head()

,Case Number,Offense Date,Court Location,Charge,Charge/Crime Type,Disposition,Chapter,Section,Paragraph,JuvenileC,...,sex,murder,Expungeable,Offenses_per_case,Inc_Expungeable_Attempts_Are,guilty,Incident_Guilty_or_missing,OffenseMatch,F/M,Offense Code
0,14-01-479818,2013-12-30,SOM,"LICENSE SUSPENDED, OP MV WITH c90 Â§23",Drugs/Distribution/Possession with Intent,GUILTY FILED,90,23,D,False,...,0,0.0,Yes,2,True,True,True,LICENSESUSPENDEDOPMVWITHC9023,M-,90/23/D
1,14-01-479826,2013-12-29,LOW,"LICENSE SUSPENDED, OP MV WITH c90 Â§23",Drugs/Possession,NOLLE PROSEQUI,90,23,D,False,...,0,0.0,Yes,6,True,False,False,LICENSESUSPENDEDOPMVWITHC9023,M-,90/23/D
2,14-01-479854,2013-12-31,SOM,"LICENSE SUSPENDED, OP MV WITH c90 Â§23",Motor Vehicle Other,DISMISSED W/O PREJUDICE,90,23,D,False,...,0,0.0,Yes,3,True,False,False,LICENSESUSPENDEDOPMVWITHC9023,M-,90/23/D
3,14-01-479887,2013-12-28,AYE,"LICENSE SUSPENDED, OP MV WITH c90 Â§23",Motor Vehicle Other,DISMISSED W/O PREJUDICE,90,23,D,False,...,0,0.0,Yes,2,True,False,True,LICENSESUSPENDEDOPMVWITHC9023,M-,90/23/D
4,14-01-479888,2013-05-19,CON,"LICENSE SUSPENDED, OP MV WITH c90 Â§23",Motor Vehicle Other,DISMISSED BY FINES,90,23,D,False,...,0,0.0,Yes,2,True,False,True,LICENSESUSPENDEDOPMVWITHC9023,M-,90/23/D


In [48]:
fm_nan_df = merged_df.loc[pd.isna(merged_df["F/M"])]
fm_nan_df.head()

,Case Number,Offense Date,Court Location,Charge,Charge/Crime Type,Disposition,Chapter,Section,Paragraph,JuvenileC,...,sex,murder,Expungeable,Offenses_per_case,Inc_Expungeable_Attempts_Are,guilty,Incident_Guilty_or_missing,OffenseMatch,F/M,Offense Code
118492,14-01-479829,2013-12-05,NAT,CONSPIRACY c274 Â§7,Larceny Other,DISMISSED W/O PREJUDICE,274,7,NaN,False,...,0,0.0,Yes,2,True,False,False,CONSPIRACYC2747,NaN,274/7
118493,14-01-480057,2012-12-28,LOW,CONSPIRACY c274 Â§7,Larceny Other,DISMISSED OUTRIGHT AT REQUEST OF CW (POST-ARR),274,7,NaN,False,...,0,0.0,Yes,2,True,False,False,CONSPIRACYC2747,NaN,274/7
118494,14-01-480081,2013-07-17,WAL,CONSPIRACY c274 Â§7,Larceny from Building,NOLLE PROSEQUI,274,7,NaN,False,...,0,0.0,Yes,4,True,False,False,CONSPIRACYC2747,NaN,274/7
118495,14-01-480273,2013-12-26,NAT,CONSPIRACY c274 Â§7,Larceny Other,DISMISSED W/O PREJUDICE,274,7,NaN,False,...,0,0.0,Yes,3,True,False,False,CONSPIRACYC2747,NaN,274/7
118496,14-01-480415,2013-12-04,MAL,CONSPIRACY c274 Â§7,Larceny Other,GUILTY CHANGE OF PLEA,274,7,NaN,False,...,0,0.0,Yes,2,True,True,True,CONSPIRACYC2747,NaN,274/7


In [49]:
nomatch_df = merged_df.loc[pd.isna(merged_df["Offense Code"])]
nomatch_df.head()

,Case Number,Offense Date,Court Location,Charge,Charge/Crime Type,Disposition,Chapter,Section,Paragraph,JuvenileC,...,sex,murder,Expungeable,Offenses_per_case,Inc_Expungeable_Attempts_Are,guilty,Incident_Guilty_or_missing,OffenseMatch,F/M,Offense Code
148521,14-01-479832,2013-08-07,AYE,"STOP/YIELD, FAIL TO * c89 Â§9",Motor Vehicle Other,RESPONSIBLE,89,9,NaN,False,...,0,0.0,Yes,2,True,True,True,STOPYIELDFAILTOC899,NaN,NaN
148522,14-01-479966,2013-12-14,LOW,"STOP/YIELD, FAIL TO * c89 Â§9",Motor Vehicle Other,RESPONSIBLE,89,9,NaN,False,...,0,0.0,Yes,2,True,True,True,STOPYIELDFAILTOC899,NaN,NaN
148523,14-01-480004,2013-04-26,LOW,"STOP/YIELD, FAIL TO * c89 Â§9",Motor Vehicle Other,NOT RESPONSIBLE,89,9,NaN,False,...,0,0.0,Yes,2,True,False,False,STOPYIELDFAILTOC899,NaN,NaN
148524,14-01-480032,2013-05-31,LOW,"STOP/YIELD, FAIL TO * c89 Â§9",Motor Vehicle Other,DISMISSED W/O PREJUDICE,89,9,NaN,False,...,0,0.0,Yes,3,True,False,False,STOPYIELDFAILTOC899,NaN,NaN
148525,14-01-480044,2013-12-21,LOW,"STOP/YIELD, FAIL TO * c89 Â§9",Motor Vehicle Other,RESPONSIBLE,89,9,NaN,False,...,0,0.0,Yes,3,True,True,True,STOPYIELDFAILTOC899,NaN,NaN


In [50]:
cNorthWestern_df = pd.read_csv("../data/cleaned/clean_northwestern.csv")
clean_suff_df = pd.read_csv("../data/cleaned/clean_suffolk.csv")